# Text classification with an RNN


### 뉴스 분석
* 1986년에 로이터에서 공개한 짧은 뉴스 기사와 토픽의 집합인 로이터 데이터셋을 사용


## Setup

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [ ]:
import numpy as np

from keras.datasets import reuters

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.preprocessing import sequence
from keras.utils import np_utils

## DataSet

* keras에서 제공되는 reuters datasets  이용
* 46개의 토픽,  각 토픽은 학습 세트에 최소한 10개의 샘플로 구성
* 8,982개의 학습 샘플과 2,246개의 테스트 샘플로 구성

In [ ]:
# num_words=10000, 데이터에서 가장 자주 등장하는 단어 1만 개 로 제한

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)
     

In [ ]:
#data size, data 내용 확인
print("train data =", len(train_data))
print("test data = ", len(test_data))
print("train_data[0]=", train_data[0])

In [ ]:
#encoding, decoding 
word_index = reuters.get_word_index()
print(word_index['happy'])

indexWord = {}
for key, value in word_index.items():
 indexWord[value] = key
print(indexWord[100])

s = ""
for x in  train_data[0]:
    s += indexWord[x] + " "
print("train_data[0]=",s)            

In [ ]:

def encode(str):
    slist = str.split()
    s = []
    for w in slist:
        s.append(word_index[w])
    return(s)    

def decode(elist):
    s = ""
    for c in elist:
        print(c)
        s += indexWord[c] + " "
    return(s)    

s = "i have a dream"
es = encode(s)
print(es)
ds = decode(es)
print(ds)


## 학습을 위한 데이터 전처리

In [ ]:
# 데이터를 pad seauence로 변환
X_train = sequence.pad_sequences(train_data, maxlen=100)
X_test = sequence.pad_sequences(test_data, maxlen=100)
# 레이블을 범주형으로 변환
y_train = np_utils.to_categorical(train_labels)
y_test = np_utils.to_categorical(test_labels)

## 모델 생성

In [ ]:
#mode 구성
model = Sequential([
    Embedding(1000, 120),
    LSTM(120),
    Dense(46, activation='softmax')])
    
model.compile(loss='categorical_crossentropy', optimizer='adam',
 metrics=['accuracy'])    

## Train the model

In [ ]:
history = model.fit(X_train, y_train, batch_size=100,  epochs=20, validation_data=(X_test, y_test))

In [ ]:
print('\n 테스트 정확도 : %.4f' % (model.evaluate(X_test, y_test)[1]))

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
  plt.show()

In [ ]:
plot_graphs(history, 'accuracy')

In [ ]:
plot_graphs(history, 'loss')